# Process Data
- Color Information
- Embedding Information
- Object Detection
- Captioning

In [ ]:
from Museum import Museum
from params.collections import MUSEUMS

### Color Palette

In [ ]:
for name,info in MUSEUMS.items():
  print("color:", name)
  Museum.get_colors(info)

### Embeddings (CLIP)

In [ ]:
for name,info in MUSEUMS.items():
  print("embeddings:", name)
  Museum.get_embeddings(info, "clip")

### Embeddings (SigLip2)

In [ ]:
for name,info in MUSEUMS.items():
  print("embeddings:", name)
  Museum.get_embeddings(info, "siglip2")

### Objects (Owlv2)

In [ ]:
for name,info in MUSEUMS.items():
  print("objects:", name)
  Museum.get_objects(info)

### Export Object Crop Images

In [ ]:
for name,info in MUSEUMS.items():
  print("export crops:", name)
  Museum.export_object_crops(info)

### Captions (Llama3.2-vision)

In [ ]:
for name,info in MUSEUMS.items():
  print("caption:", name)
  Museum.get_captions(info, model="llama3.2-vision:11b")

### Captions (Gemma3)

In [ ]:
for name,info in MUSEUMS.items():
  print("caption:", name)
  Museum.get_captions(info, model="gemma3:4b")

### Combine and Export JSONs

In [ ]:
for name,info in MUSEUMS.items():
  print("combine:", name)
  Museum.combine_data(info)

## Combine Data

### Combine all museum JSONs (and add tsne embeddings)

In [ ]:
from Museum import Museum
from params.collections import MUSEUMS

from utils.data_utils import get_tsne_embeddings

OUT_PREFIX = "20250705"
OUT_DIR = "./metadata/json"
DATA_DIRS = ["processed"]

embedding_data = Museum.combine_all_data(MUSEUMS, "embeddings")
tsne_embeddings = get_tsne_embeddings(embedding_data)

extra_data = {
  "processed" : tsne_embeddings
}

Museum.combine_museums(MUSEUMS, OUT_DIR, OUT_PREFIX, DATA_DIRS, extra_data)

### Add image ratios

In [ ]:
import json

from utils.data_utils import get_image_ratios

OUT_PREFIX = "20250705"
OUT_DIR = "./metadata/json"
IMGS_DIR = "../../imgs/arts/100"
DATA_FILE = f"{OUT_DIR}/{OUT_PREFIX}_processed.json"

ratioH_data = get_image_ratios(IMGS_DIR)

with open(DATA_FILE, "r", encoding="utf-8") as ifp:
  all_data = json.load(ifp)

for k,v in all_data.items():
  if k in ratioH_data:
    all_data[k]["image"] = {
      "image": all_data[k]["image"],
      "ratio": ratioH_data[k]
    }
  else:
    print(k, "not in ratioH_data")

with open(DATA_FILE, "w", encoding="utf-8") as ofp:
  json.dump(all_data, ofp, separators=(",",":"), sort_keys=True, ensure_ascii=False)

### ~~Combine all museum JSONs~~

In [ ]:
from Museum import Museum
from params.collections import MUSEUMS

OUT_PREFIX = "20250705"
OUT_DIR = "./metadata/json"
DATA_DIRS = ["processed"]

Museum.combine_museums(MUSEUMS, OUT_DIR, OUT_PREFIX, DATA_DIRS)

### Export cluster information (with cluster descriptions using Gemma3 and SigLip2)

In [ ]:
from Museum import Museum

from params.collections import MUSEUMS
from utils.data_utils import Clusterer

OUT_PREFIX = "20250705"
IMAGES_PATH = "../../imgs/arts/500"

embedding_data = Museum.combine_all_data(MUSEUMS, "embeddings")

Clusterer(embedding_data, OUT_PREFIX, IMAGES_PATH).export_clusters("clusters.json")

### Export to HuggingFace

In [ ]:
import json
from Museum import Museum
from params.collections import MUSEUMS

from datasets import Dataset
from huggingface_hub import login as hf_login

PREFIX = "20250705"
DATA_DIR = "./metadata/json"
DATA_FILE = f"{DATA_DIR}/{PREFIX}_processed.json"
CLUSTER_FILE = f"{DATA_DIR}/{PREFIX}_clusters.json"

embedding_data = Museum.combine_all_data(MUSEUMS, "embeddings")

with open(DATA_FILE, "r", encoding="utf-8") as ifp:
  all_data = json.load(ifp)

with open(CLUSTER_FILE, "r", encoding="utf-8") as ifp:
  cluster_data_all = json.load(ifp)
  cluster_data = cluster_data_all["8"]["images"]


for id in all_data.keys():
  for emb in embedding_data[id].keys():
    all_data[id]["embeddings"][emb] = embedding_data[id][emb]
  all_data[id]["clusters"] = {
    "distances": cluster_data[id]["distances"],
  }
  all_data[id]["id"] = str(all_data[id]["id"])

In [ ]:
hf_login()

In [ ]:
ds = Dataset.from_list(list(all_data.values()))
ds.push_to_hub("acervos-digitais/meta-acervos-data")